In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import re

In [2]:
sheets_to_load = ['Germany',
                  'Italy',
                  'Poland',
                  'Switzerland',
                  'Sweden',
                  'Thailand',
                  'UK',
                  'Vietnam',
                  'Australia']

# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\SEPTEMBER\q18-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\OCTOBER\q18-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\NOVEMBER\q18-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\DECEMBER\q18-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\2025\JANUARY\q18-compilation.xlsx', sheet_name=sheets_to_load)
selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\2025\JULY\q18-compilation.xlsx', sheet_name=sheets_to_load)

Germany = selected_sheets['Germany']
Italy = selected_sheets['Italy']
Poland = selected_sheets['Poland']
Switzerland = selected_sheets['Switzerland']
Sweden = selected_sheets['Sweden']
Thailand = selected_sheets['Thailand']
UK = selected_sheets['UK']
Vietnam = selected_sheets['Vietnam']
Australia = selected_sheets['Australia']

### Germany

In [3]:
Germany.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), object(1)
memory usage: 528.3+ KB


In [4]:
Germany.head(2) #initial dataset

,Response ID,"Refrigerator:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,GER2001,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GER2002,1.0,2.0,NaN,4.0,5.0,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN


In [5]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_GM = Germany.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_GM['Product_Category_cleaned'] = unpivoted_GM['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_GM = unpivoted_GM[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_GM['Product_Category_cleaned'] = unpivoted_GM['Product_Category_cleaned'].str.replace(
    ":GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_GM.tail())

      Response ID                                   Product_Category  \
67195     GER2396  None of these:ROWENTA:GM - Which product categ...   
67196     GER2397  None of these:ROWENTA:GM - Which product categ...   
67197     GER2398  None of these:ROWENTA:GM - Which product categ...   
67198     GER2399  None of these:ROWENTA:GM - Which product categ...   
67199     GER2400  None of these:ROWENTA:GM - Which product categ...   

      Product_Category_cleaned  Seen or Heard Y/N  
67195    None of these:ROWENTA                NaN  
67196    None of these:ROWENTA                NaN  
67197    None of these:ROWENTA                NaN  
67198    None of these:ROWENTA                NaN  
67199    None of these:ROWENTA                NaN  


In [6]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_GM['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_GM['Brand'] = brand_names
unpivoted_GM['Ad_Type'] = ad_types

In [7]:
count = unpivoted_GM['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
2.0     753
5.0     668
1.0     538
4.0     442
3.0     418
8.0     398
99.0    381
10.0    378
9.0     317
12.0    311
13.0    257
7.0     231
11.0    227
6.0     217
Name: count, dtype: int64


In [8]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_GM = unpivoted_GM.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_GM = unpivoted_GM.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_GM = unpivoted_GM[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_GM['Seen or Heard Y/N'] = np.where(unpivoted_GM['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_GM['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_GM['Seen or Heard Y/N'].isna(), 0, unpivoted_GM['Seen or Heard Y/N'])))

In [9]:
unpivoted_GM.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,GER2001,Refrigerator:BOSCH:GM - Which product categori...,Refrigerator,0.0,BOSCH
1,GER2002,Refrigerator:BOSCH:GM - Which product categori...,Refrigerator,1.0,BOSCH
2,GER2003,Refrigerator:BOSCH:GM - Which product categori...,Refrigerator,0.0,BOSCH
3,GER2004,Refrigerator:BOSCH:GM - Which product categori...,Refrigerator,0.0,BOSCH
4,GER2005,Refrigerator:BOSCH:GM - Which product categori...,Refrigerator,0.0,BOSCH


In [10]:
count = unpivoted_GM['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_GM['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_GM)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     61664
1.0      5155
99.0      381
Name: count, dtype: int64


Brand
BOSCH        5600
MIELE        5600
SIEMENS      5600
AEG          5600
SAMSUNG      5600
BAUKNECHT    5600
PHILIPS      5600
LG           5600
DYSON        5600
NEFF         5600
HAIER        5600
ROWENTA      5600
Name: count, dtype: int64


Total number of rows: 67200


### Sweden

In [11]:
Sweden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), object(1)
memory usage: 528.3+ KB


In [12]:
Sweden.head(2) #initial dataset

,Response ID,"Refrigerator:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,SWE2001,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SWE2002,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_SW = Sweden.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_SW['Product_Category_cleaned'] = unpivoted_SW['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_SW = unpivoted_SW[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_SW['Product_Category_cleaned'] = unpivoted_SW['Product_Category_cleaned'].str.replace(
    ":SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_SW.tail())

      Response ID                                   Product_Category  \
67195     SWE2396  None of these:ROBOROCK:SW - Which product cate...   
67196     SWE2397  None of these:ROBOROCK:SW - Which product cate...   
67197     SWE2398  None of these:ROBOROCK:SW - Which product cate...   
67198     SWE2399  None of these:ROBOROCK:SW - Which product cate...   
67199     SWE2400  None of these:ROBOROCK:SW - Which product cate...   

      Product_Category_cleaned  Seen or Heard Y/N  
67195   None of these:ROBOROCK                NaN  
67196   None of these:ROBOROCK                NaN  
67197   None of these:ROBOROCK                NaN  
67198   None of these:ROBOROCK                NaN  
67199   None of these:ROBOROCK                NaN  


In [14]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_SW['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SW['Brand'] = brand_names
unpivoted_SW['Ad_Type'] = ad_types

In [15]:
count = unpivoted_SW['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
2.0     690
1.0     583
5.0     555
4.0     449
3.0     375
99.0    363
10.0    350
8.0     341
12.0    301
9.0     272
13.0    237
11.0    236
6.0     191
7.0     165
Name: count, dtype: int64


In [16]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_SW = unpivoted_SW.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_SW = unpivoted_SW.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_SW = unpivoted_SW[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_SW['Seen or Heard Y/N'] = np.where(unpivoted_SW['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_SW['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_SW['Seen or Heard Y/N'].isna(), 0, unpivoted_SW['Seen or Heard Y/N'])))

In [17]:
unpivoted_SW.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,SWE2001,Refrigerator:ELECTROLUX:SW - Which product cat...,Refrigerator,0.0,ELECTROLUX
1,SWE2002,Refrigerator:ELECTROLUX:SW - Which product cat...,Refrigerator,1.0,ELECTROLUX
2,SWE2003,Refrigerator:ELECTROLUX:SW - Which product cat...,Refrigerator,0.0,ELECTROLUX
3,SWE2004,Refrigerator:ELECTROLUX:SW - Which product cat...,Refrigerator,1.0,ELECTROLUX
4,SWE2005,Refrigerator:ELECTROLUX:SW - Which product cat...,Refrigerator,0.0,ELECTROLUX


In [18]:
count = unpivoted_SW['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SW['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SW)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     62092
1.0      4745
99.0      363
Name: count, dtype: int64


Brand
ELECTROLUX    5600
BOSCH         5600
SAMSUNG       5600
MIELE         5600
PHILIPS       5600
SIEMENS       5600
CYLINDA       5600
WHIRLPOOL     5600
LG            5600
DYSON         5600
AEG           5600
ROBOROCK      5600
Name: count, dtype: int64


Total number of rows: 67200


### Italy

In [19]:
Italy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), object(1)
memory usage: 528.3+ KB


In [20]:
Italy.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,ITA2001,1.0,NaN,NaN,4.0,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,8.0,9.0,10.0,NaN,NaN,NaN,NaN
1,ITA2002,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,8.0,NaN,10.0,NaN,12.0,NaN,NaN


In [21]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_IT = Italy.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_IT['Product_Category_cleaned'] = unpivoted_IT['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_IT = unpivoted_IT[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_IT['Product_Category_cleaned'] = unpivoted_IT['Product_Category_cleaned'].str.replace(
    ":IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_IT.tail())

      Response ID                                   Product_Category  \
67195     ITA2396  None of these:SIEMENS:IT - Which product categ...   
67196     ITA2397  None of these:SIEMENS:IT - Which product categ...   
67197     ITA2398  None of these:SIEMENS:IT - Which product categ...   
67198     ITA2399  None of these:SIEMENS:IT - Which product categ...   
67199     ITA2400  None of these:SIEMENS:IT - Which product categ...   

      Product_Category_cleaned  Seen or Heard Y/N  
67195    None of these:SIEMENS                NaN  
67196    None of these:SIEMENS                NaN  
67197    None of these:SIEMENS                NaN  
67198    None of these:SIEMENS                NaN  
67199    None of these:SIEMENS                NaN  


In [22]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_IT['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_IT['Brand'] = brand_names
unpivoted_IT['Ad_Type'] = ad_types

In [23]:
count = unpivoted_IT['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
2.0     840
1.0     634
4.0     564
5.0     519
3.0     496
8.0     460
12.0    345
9.0     331
7.0     309
10.0    297
99.0    283
6.0     269
13.0    224
11.0    214
Name: count, dtype: int64


In [24]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_IT = unpivoted_IT.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_IT = unpivoted_IT.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_IT = unpivoted_IT[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_IT['Seen or Heard Y/N'] = np.where(unpivoted_IT['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_IT['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_IT['Seen or Heard Y/N'].isna(), 0, unpivoted_IT['Seen or Heard Y/N'])))

In [25]:
unpivoted_IT.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,ITA2001,Refrigerator:SAMSUNG:IT - Which product catego...,Refrigerator,1.0,SAMSUNG
1,ITA2002,Refrigerator:SAMSUNG:IT - Which product catego...,Refrigerator,0.0,SAMSUNG
2,ITA2003,Refrigerator:SAMSUNG:IT - Which product catego...,Refrigerator,0.0,SAMSUNG
3,ITA2004,Refrigerator:SAMSUNG:IT - Which product catego...,Refrigerator,0.0,SAMSUNG
4,ITA2005,Refrigerator:SAMSUNG:IT - Which product catego...,Refrigerator,0.0,SAMSUNG


In [26]:
count = unpivoted_IT['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_IT['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_IT)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     61415
1.0      5502
99.0      283
Name: count, dtype: int64


Brand
SAMSUNG       5600
BOSCH         5600
WHIRLPOOL     5600
LG            5600
DYSON         5600
MIELE         5600
ELECTROLUX    5600
HOTPOINT      5600
BEKO          5600
AEG           5600
HAIER         5600
SIEMENS       5600
Name: count, dtype: int64


Total number of rows: 67200


### UK

In [27]:
UK.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), object(1)
memory usage: 528.3+ KB


In [28]:
UK.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,UK2001,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UK2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_UK = UK.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_UK['Product_Category_cleaned'] = unpivoted_UK['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_UK = unpivoted_UK[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_UK['Product_Category_cleaned'] = unpivoted_UK['Product_Category_cleaned'].str.replace(
    ":UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_UK.tail())

      Response ID                                   Product_Category  \
67195      UK2396  None of these:NEFF:UK - Which product categori...   
67196      UK2397  None of these:NEFF:UK - Which product categori...   
67197      UK2398  None of these:NEFF:UK - Which product categori...   
67198      UK2399  None of these:NEFF:UK - Which product categori...   
67199      UK2400  None of these:NEFF:UK - Which product categori...   

      Product_Category_cleaned  Seen or Heard Y/N  
67195       None of these:NEFF                NaN  
67196       None of these:NEFF               99.0  
67197       None of these:NEFF                NaN  
67198       None of these:NEFF                NaN  
67199       None of these:NEFF               99.0  


In [30]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_UK['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_UK['Brand'] = brand_names
unpivoted_UK['Ad_Type'] = ad_types

In [31]:
count = unpivoted_UK['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
2.0     734
1.0     581
5.0     537
4.0     413
99.0    410
8.0     390
3.0     373
10.0    343
12.0    309
9.0     237
7.0     237
13.0    223
6.0     213
11.0    203
Name: count, dtype: int64


In [32]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_UK = unpivoted_UK.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_UK = unpivoted_UK.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_UK = unpivoted_UK[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_UK['Seen or Heard Y/N'] = np.where(unpivoted_UK['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_UK['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_UK['Seen or Heard Y/N'].isna(), 0, unpivoted_UK['Seen or Heard Y/N'])))

In [33]:
unpivoted_UK.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,UK2001,Refrigerator:SAMSUNG:UK - Which product catego...,Refrigerator,0.0,SAMSUNG
1,UK2002,Refrigerator:SAMSUNG:UK - Which product catego...,Refrigerator,0.0,SAMSUNG
2,UK2003,Refrigerator:SAMSUNG:UK - Which product catego...,Refrigerator,0.0,SAMSUNG
3,UK2004,Refrigerator:SAMSUNG:UK - Which product catego...,Refrigerator,1.0,SAMSUNG
4,UK2005,Refrigerator:SAMSUNG:UK - Which product catego...,Refrigerator,0.0,SAMSUNG


In [34]:
count = unpivoted_UK['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_UK['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_UK)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     61997
1.0      4793
99.0      410
Name: count, dtype: int64


Brand
SAMSUNG     5600
LG          5600
HOTPOINT    5600
BOSCH       5600
DYSON       5600
BEKO        5600
SHARK       5600
ZANUSSI     5600
MIELE       5600
AEG         5600
SIEMENS     5600
NEFF        5600
Name: count, dtype: int64


Total number of rows: 67200


### Poland

In [35]:
Poland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), object(1)
memory usage: 528.3+ KB


In [36]:
Poland.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,POL2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POL2002,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_PO = Poland.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_PO['Product_Category_cleaned'] = unpivoted_PO['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_PO = unpivoted_PO[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_PO['Product_Category_cleaned'] = unpivoted_PO['Product_Category_cleaned'].str.replace(
    ":PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_PO.tail())

      Response ID                                   Product_Category  \
67195     POL2396  None of these:AEG:PO - Which product categorie...   
67196     POL2397  None of these:AEG:PO - Which product categorie...   
67197     POL2398  None of these:AEG:PO - Which product categorie...   
67198     POL2399  None of these:AEG:PO - Which product categorie...   
67199     POL2400  None of these:AEG:PO - Which product categorie...   

      Product_Category_cleaned  Seen or Heard Y/N  
67195        None of these:AEG                NaN  
67196        None of these:AEG                NaN  
67197        None of these:AEG                NaN  
67198        None of these:AEG                NaN  
67199        None of these:AEG                NaN  


In [38]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_PO['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_PO['Brand'] = brand_names
unpivoted_PO['Ad_Type'] = ad_types

In [39]:
count = unpivoted_PO['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
2.0     773
1.0     680
5.0     540
8.0     476
4.0     445
3.0     379
10.0    367
9.0     340
12.0    283
7.0     260
11.0    246
99.0    240
13.0    217
6.0     193
Name: count, dtype: int64


In [40]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_PO = unpivoted_PO.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_PO = unpivoted_PO.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_PO = unpivoted_PO[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_PO['Seen or Heard Y/N'] = np.where(unpivoted_PO['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_PO['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_PO['Seen or Heard Y/N'].isna(), 0, unpivoted_PO['Seen or Heard Y/N'])))

In [41]:
unpivoted_PO.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,POL2001,Refrigerator:SAMSUNG:PO - Which product catego...,Refrigerator,0.0,SAMSUNG
1,POL2002,Refrigerator:SAMSUNG:PO - Which product catego...,Refrigerator,0.0,SAMSUNG
2,POL2003,Refrigerator:SAMSUNG:PO - Which product catego...,Refrigerator,0.0,SAMSUNG
3,POL2004,Refrigerator:SAMSUNG:PO - Which product catego...,Refrigerator,0.0,SAMSUNG
4,POL2005,Refrigerator:SAMSUNG:PO - Which product catego...,Refrigerator,0.0,SAMSUNG


In [42]:
count = unpivoted_PO['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_PO['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_PO)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     61761
1.0      5199
99.0      240
Name: count, dtype: int64


Brand
SAMSUNG       5600
BOSCH         5600
ELECTROLUX    5600
BEKO          5600
AMICA         5600
PHILIPS       5600
WHIRLPOOL     5600
LG            5600
DYSON         5600
HAIER         5600
MIELE         5600
AEG           5600
Name: count, dtype: int64


Total number of rows: 67200


### Thailand

In [43]:
Thailand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), object(1)
memory usage: 528.3+ KB


In [44]:
Thailand.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,TH2001,1.0,2.0,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN
1,TH2002,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_TH = Thailand.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_TH['Product_Category_cleaned'] = unpivoted_TH['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_TH = unpivoted_TH[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_TH['Product_Category_cleaned'] = unpivoted_TH['Product_Category_cleaned'].str.replace(
    ":TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_TH.tail())

      Response ID                                   Product_Category  \
67195      TH2396  None of these:DYSON:TH - Which product categor...   
67196      TH2397  None of these:DYSON:TH - Which product categor...   
67197      TH2398  None of these:DYSON:TH - Which product categor...   
67198      TH2399  None of these:DYSON:TH - Which product categor...   
67199      TH2400  None of these:DYSON:TH - Which product categor...   

      Product_Category_cleaned  Seen or Heard Y/N  
67195      None of these:DYSON                NaN  
67196      None of these:DYSON                NaN  
67197      None of these:DYSON                NaN  
67198      None of these:DYSON                NaN  
67199      None of these:DYSON                NaN  


In [46]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_TH['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_TH['Brand'] = brand_names
unpivoted_TH['Ad_Type'] = ad_types

In [47]:
count = unpivoted_TH['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
1.0     1250
2.0     1167
7.0     1085
5.0      837
10.0     823
12.0     777
6.0      699
13.0     623
3.0      565
8.0      551
11.0     528
4.0      494
9.0      478
99.0     173
Name: count, dtype: int64


In [48]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_TH = unpivoted_TH.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_TH = unpivoted_TH.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_TH = unpivoted_TH[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_TH['Seen or Heard Y/N'] = np.where(unpivoted_TH['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_TH['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_TH['Seen or Heard Y/N'].isna(), 0, unpivoted_TH['Seen or Heard Y/N'])))

In [49]:
unpivoted_TH.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,TH2001,Refrigerator:SAMSUNG:TH - Which product catego...,Refrigerator,1.0,SAMSUNG
1,TH2002,Refrigerator:SAMSUNG:TH - Which product catego...,Refrigerator,0.0,SAMSUNG
2,TH2003,Refrigerator:SAMSUNG:TH - Which product catego...,Refrigerator,1.0,SAMSUNG
3,TH2004,Refrigerator:SAMSUNG:TH - Which product catego...,Refrigerator,0.0,SAMSUNG
4,TH2005,Refrigerator:SAMSUNG:TH - Which product catego...,Refrigerator,0.0,SAMSUNG


In [50]:
count = unpivoted_TH['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_TH['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_TH)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     57150
1.0      9877
99.0      173
Name: count, dtype: int64


Brand
SAMSUNG       5600
LG            5600
MITSUBISHI    5600
ELECTROLUX    5600
PANASONIC     5600
SHARP         5600
PHILIPS       5600
HITACHI       5600
HAIER         5600
TEFAL         5600
BOSCH         5600
DYSON         5600
Name: count, dtype: int64


Total number of rows: 67200


### Vietnam

In [51]:
Vietnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), object(1)
memory usage: 528.3+ KB


In [52]:
Vietnam.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,VN02001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VN02002,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_VN = Vietnam.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_VN['Product_Category_cleaned'] = unpivoted_VN['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_VN = unpivoted_VN[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_VN['Product_Category_cleaned'] = unpivoted_VN['Product_Category_cleaned'].str.replace(
    ":VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_VN.tail())

      Response ID                                   Product_Category  \
67195     VN02396  None of these:MIDEA:VN - Which product categor...   
67196     VN02397  None of these:MIDEA:VN - Which product categor...   
67197     VN02398  None of these:MIDEA:VN - Which product categor...   
67198     VN02399  None of these:MIDEA:VN - Which product categor...   
67199     VN02400  None of these:MIDEA:VN - Which product categor...   

      Product_Category_cleaned  Seen or Heard Y/N  
67195      None of these:MIDEA                NaN  
67196      None of these:MIDEA                NaN  
67197      None of these:MIDEA                NaN  
67198      None of these:MIDEA                NaN  
67199      None of these:MIDEA                NaN  


In [54]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_VN['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_VN['Brand'] = brand_names
unpivoted_VN['Ad_Type'] = ad_types

In [55]:
count = unpivoted_VN['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
1.0     1253
2.0     1167
7.0     1068
10.0     883
5.0      847
6.0      805
12.0     765
8.0      680
13.0     679
4.0      679
11.0     670
3.0      664
9.0      633
99.0     143
Name: count, dtype: int64


In [56]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_VN = unpivoted_VN.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_VN = unpivoted_VN.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_VN = unpivoted_VN[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_VN['Seen or Heard Y/N'] = np.where(unpivoted_VN['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_VN['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_VN['Seen or Heard Y/N'].isna(), 0, unpivoted_VN['Seen or Heard Y/N'])))

In [57]:
unpivoted_VN.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,VN02001,Refrigerator:SAMSUNG:VN - Which product catego...,Refrigerator,0.0,SAMSUNG
1,VN02002,Refrigerator:SAMSUNG:VN - Which product catego...,Refrigerator,0.0,SAMSUNG
2,VN02003,Refrigerator:SAMSUNG:VN - Which product catego...,Refrigerator,0.0,SAMSUNG
3,VN02004,Refrigerator:SAMSUNG:VN - Which product catego...,Refrigerator,0.0,SAMSUNG
4,VN02005,Refrigerator:SAMSUNG:VN - Which product catego...,Refrigerator,0.0,SAMSUNG


In [58]:
count = unpivoted_VN['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_VN['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_VN)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     56264
1.0     10793
99.0      143
Name: count, dtype: int64


Brand
SAMSUNG       5600
PANASONIC     5600
LG            5600
TOSHIBA       5600
ELECTROLUX    5600
SHARP         5600
PHILIPS       5600
HITACHI       5600
XIAOMI        5600
AQUA          5600
BOSCH         5600
MIDEA         5600
Name: count, dtype: int64


Total number of rows: 67200


### Australia

In [59]:
Australia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), object(1)
memory usage: 528.3+ KB


In [60]:
Australia.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,AUS2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AUS2002,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_AU = Australia.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_AU['Product_Category_cleaned'] = unpivoted_AU['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_AU = unpivoted_AU[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_AU['Product_Category_cleaned'] = unpivoted_AU['Product_Category_cleaned'].str.replace(
    ":AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_AU.tail())

      Response ID                                   Product_Category  \
67195     AUS2396  None of these:AEG:AU - Which product categorie...   
67196     AUS2397  None of these:AEG:AU - Which product categorie...   
67197     AUS2398  None of these:AEG:AU - Which product categorie...   
67198     AUS2399  None of these:AEG:AU - Which product categorie...   
67199     AUS2400  None of these:AEG:AU - Which product categorie...   

      Product_Category_cleaned  Seen or Heard Y/N  
67195        None of these:AEG                NaN  
67196        None of these:AEG                NaN  
67197        None of these:AEG               99.0  
67198        None of these:AEG                NaN  
67199        None of these:AEG                NaN  


In [62]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_AU['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_AU['Brand'] = brand_names
unpivoted_AU['Ad_Type'] = ad_types

In [63]:
count = unpivoted_AU['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
1.0     1108
2.0     1043
7.0      833
5.0      720
10.0     638
12.0     570
4.0      545
6.0      535
8.0      519
3.0      509
13.0     503
11.0     425
9.0      396
99.0     307
Name: count, dtype: int64


In [64]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_AU = unpivoted_AU.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_AU = unpivoted_AU.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_AU = unpivoted_AU[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_AU['Seen or Heard Y/N'] = np.where(unpivoted_AU['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_AU['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_AU['Seen or Heard Y/N'].isna(), 0, unpivoted_AU['Seen or Heard Y/N'])))

In [65]:
unpivoted_AU.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,AUS2001,Refrigerator:SAMSUNG:AU - Which product catego...,Refrigerator,0.0,SAMSUNG
1,AUS2002,Refrigerator:SAMSUNG:AU - Which product catego...,Refrigerator,1.0,SAMSUNG
2,AUS2003,Refrigerator:SAMSUNG:AU - Which product catego...,Refrigerator,0.0,SAMSUNG
3,AUS2004,Refrigerator:SAMSUNG:AU - Which product catego...,Refrigerator,1.0,SAMSUNG
4,AUS2005,Refrigerator:SAMSUNG:AU - Which product catego...,Refrigerator,1.0,SAMSUNG


In [66]:
count = unpivoted_AU['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     58549
1.0      8344
99.0      307
Name: count, dtype: int64


Brand
SAMSUNG         5600
LG              5600
DYSON           5600
WESTINGHOUSE    5600
BOSCH           5600
HISENSE         5600
MIELE           5600
SMEG            5600
ELECTROLUX      5600
HAIER           5600
AEG             5600
Name: count, dtype: int64


Total number of rows: 67200


In [67]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_AU[unpivoted_AU['Brand'].isna() | (unpivoted_AU['Brand'] == '')].shape[0]
print("Count of blank values in Brand:", blank_count)

Count of blank values in Brand: 5600


In [68]:
# Replace NaN values in 'Brand' column with 'FISHER&PAYKEL'
unpivoted_AU['Brand'] = unpivoted_AU['Brand'].fillna('FISHER&PAYKEL')

# Check the count of NaN values after replacement
na_count = unpivoted_AU['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [69]:
count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Brand
SAMSUNG          5600
LG               5600
DYSON            5600
WESTINGHOUSE     5600
FISHER&PAYKEL    5600
BOSCH            5600
HISENSE          5600
MIELE            5600
SMEG             5600
ELECTROLUX       5600
HAIER            5600
AEG              5600
Name: count, dtype: int64


Total number of rows: 67200


### Switzerland

In [70]:
Switzerland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), object(1)
memory usage: 528.3+ KB


In [71]:
Switzerland.head(2) #initial dataset

,Response ID,"Refrigerator:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,SWIT2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SWIT2002,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_SZ = Switzerland.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_SZ['Product_Category_cleaned'] = unpivoted_SZ['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_SZ['Product_Category_cleaned'] = unpivoted_SZ['Product_Category_cleaned'].str.replace(
    ":SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_SZ.tail())

      Response ID                                   Product_Category  \
67195    SWIT2396  None of these:SCHULTHESS:SZ - Which product ca...   
67196    SWIT2397  None of these:SCHULTHESS:SZ - Which product ca...   
67197    SWIT2398  None of these:SCHULTHESS:SZ - Which product ca...   
67198    SWIT2399  None of these:SCHULTHESS:SZ - Which product ca...   
67199    SWIT2400  None of these:SCHULTHESS:SZ - Which product ca...   

       Product_Category_cleaned  Seen or Heard Y/N  
67195  None of these:SCHULTHESS                NaN  
67196  None of these:SCHULTHESS                NaN  
67197  None of these:SCHULTHESS                NaN  
67198  None of these:SCHULTHESS                NaN  
67199  None of these:SCHULTHESS                NaN  


In [73]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_SZ['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SZ['Brand'] = brand_names
unpivoted_SZ['Ad_Type'] = ad_types

In [74]:
count = unpivoted_SZ['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
2.0     690
1.0     578
5.0     531
4.0     473
8.0     435
3.0     405
10.0    393
99.0    390
9.0     296
12.0    247
11.0    234
6.0     217
7.0     196
13.0    194
Name: count, dtype: int64


In [75]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_SZ = unpivoted_SZ.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_SZ = unpivoted_SZ.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_SZ['Seen or Heard Y/N'] = np.where(unpivoted_SZ['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_SZ['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_SZ['Seen or Heard Y/N'].isna(), 0, unpivoted_SZ['Seen or Heard Y/N'])))

In [76]:
unpivoted_SZ.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,SWIT2001,Refrigerator:V ZUG:SZ - Which product categori...,Refrigerator,0.0,None
1,SWIT2002,Refrigerator:V ZUG:SZ - Which product categori...,Refrigerator,0.0,None
2,SWIT2003,Refrigerator:V ZUG:SZ - Which product categori...,Refrigerator,0.0,None
3,SWIT2004,Refrigerator:V ZUG:SZ - Which product categori...,Refrigerator,0.0,None
4,SWIT2005,Refrigerator:V ZUG:SZ - Which product categori...,Refrigerator,1.0,None


In [77]:
count = unpivoted_SZ['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     61921
1.0      4889
99.0      390
Name: count, dtype: int64


Brand
ELECTROLUX    5600
MIELE         5600
DYSON         5600
BOSCH         5600
SAMSUNG       5600
SIEMENS       5600
PHILIPS       5600
BAUKNECHT     5600
AEG           5600
LIEBHERR      5600
SCHULTHESS    5600
Name: count, dtype: int64


Total number of rows: 67200


In [78]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_SZ[unpivoted_SZ['Brand'].isna() | (unpivoted_SZ['Brand'] == '')].shape[0]
print("Count of blank values in 'Brand:", blank_count)

Count of blank values in 'Brand: 5600


In [79]:
# Replace NaN values in 'Brand' column with 'V Zug'
unpivoted_SZ['Brand'] = unpivoted_SZ['Brand'].fillna('V ZUG')

# Check the count of NaN values after replacement
na_count = unpivoted_SZ['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [80]:
count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Brand
V ZUG         5600
ELECTROLUX    5600
MIELE         5600
DYSON         5600
BOSCH         5600
SAMSUNG       5600
SIEMENS       5600
PHILIPS       5600
BAUKNECHT     5600
AEG           5600
LIEBHERR      5600
SCHULTHESS    5600
Name: count, dtype: int64


Total number of rows: 67200


### Concat and save them in another csv file

In [81]:
# Concatenate the melted DataFrames
combined_q18 = pd.concat([unpivoted_GM, unpivoted_SW, unpivoted_TH, unpivoted_IT, unpivoted_PO, 
                                unpivoted_SZ, unpivoted_VN, unpivoted_AU, unpivoted_UK], ignore_index=True)


# Define conditions for categorization
conditions = [
    combined_q18['Product_Category_cleaned'].isin(["Washing Machine", "Tumble Dryers", "Dishwashers"]),
    combined_q18['Product_Category_cleaned'].isin(["Refrigerator", "Oven", "Hob", "Cooker", "Hood", "Microwave"]),
    combined_q18['Product_Category_cleaned'].isin(["Vacuum Cleaners", "Air Purifier", "Air Conditioner", "Air Fryers"])
]

# Define corresponding outputs for the conditions
outputs = ["Care", "Taste", "Wellbeing & SDA"]

# Assign the new column 'Experience Area' based on conditions
combined_q18['Experience_Area'] = np.select(conditions, outputs, default="Other")

# Display the result
print(combined_q18.head())

# combined_q18.dropna(subset=['Response ID'], inplace=True)

combined_q18 = combined_q18.sort_values(by='Response ID', ascending=True)

# combined_q18.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\SEPTEMBER\combined_q18.xlsx', float_format='%.0f', index=False)
# combined_q18.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\OCTOBER\combined_q18.xlsx', float_format='%.0f', index=False)
# combined_q18.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\NOVEMBER\combined_q18.xlsx', float_format='%.0f', index=False)
# combined_q18.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\DECEMBER\combined_q18.xlsx', float_format='%.0f', index=False)
# combined_q18.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\2025\JANUARY\combined_q18.xlsx', float_format='%.0f', index=False)
combined_q18.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\2025\JULY\combined_q18.xlsx', float_format='%.0f', index=False)

print(combined_q18.head(2))

  Response ID                                   Product_Category  \
0     GER2001  Refrigerator:BOSCH:GM - Which product categori...   
1     GER2002  Refrigerator:BOSCH:GM - Which product categori...   
2     GER2003  Refrigerator:BOSCH:GM - Which product categori...   
3     GER2004  Refrigerator:BOSCH:GM - Which product categori...   
4     GER2005  Refrigerator:BOSCH:GM - Which product categori...   

  Product_Category_cleaned  Seen or Heard Y/N  Brand Experience_Area  
0             Refrigerator                0.0  BOSCH           Taste  
1             Refrigerator                1.0  BOSCH           Taste  
2             Refrigerator                0.0  BOSCH           Taste  
3             Refrigerator                0.0  BOSCH           Taste  
4             Refrigerator                0.0  BOSCH           Taste  
       Response ID                                   Product_Category  \
470400     AUS2001  Refrigerator:SAMSUNG:AU - Which product catego...   
496400     AUS2001 

In [82]:
# Count blank (NaN) rows in the 'Response ID' column
blank_count = combined_q18['Response ID'].isna().sum()
print(f"Number of blank rows in 'Response ID': {blank_count}")

Number of blank rows in 'Response ID': 0


In [83]:
combined_q18.info()

<class 'pandas.core.frame.DataFrame'>
Index: 604800 entries, 470400 to 436799
Data columns (total 6 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Response ID               604800 non-null  object 
 1   Product_Category          604800 non-null  object 
 2   Product_Category_cleaned  604800 non-null  object 
 3   Seen or Heard Y/N         604800 non-null  float64
 4   Brand                     604800 non-null  object 
 5   Experience_Area           604800 non-null  object 
dtypes: float64(1), object(5)
memory usage: 32.3+ MB


In [84]:
count = combined_q18['Experience_Area'].value_counts()
print(count)

Experience_Area
Taste              259200
Wellbeing & SDA    172800
Care               129600
Other               43200
Name: count, dtype: int64
